In [3]:
# ==============================================
# STEP 1: Install Dependencies
# ==============================================
!pip -q install --upgrade diffusers==0.29.2 transformers accelerate safetensors \
  imageio[ffmpeg] gradio torch torchvision torchaudio

import torch, imageio, numpy as np, gradio as gr, os
from diffusers import AutoPipelineForText2Image, DiffusionPipeline

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs("outputs", exist_ok=True)
print("✅ Running on:", device)

# ==============================================
# STEP 2: Load Models (Image + Video)
# ==============================================
pipe_img = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-2-1", torch_dtype=torch.float32
).to(device)

pipe_vid = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float32
).to(device)

# ==============================================
# STEP 3: Define Generation Functions
# ==============================================
def generate_image(prompt):
    image = pipe_img(prompt, num_inference_steps=20, guidance_scale=7.5).images[0]
    path = "outputs/generated_image.png"
    image.save(path)
    return path

def generate_video(prompt):
    video_frames = pipe_vid(
        prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        num_frames=16,
        height=320,
        width=576
    ).frames[0]

    path = "outputs/generated_video.mp4"
    imageio.mimsave(path, [np.array(f) for f in video_frames], fps=8, quality=8)
    return path

# ==============================================
# STEP 4: Gradio UI (Prompt → Image/Video)
# ==============================================
def run(prompt, mode):
    if mode == "Image":
        return generate_image(prompt)
    else:
        return generate_video(prompt)

demo = gr.Interface(
    fn=run,
    inputs=[
        gr.Textbox(label="Enter your prompt here 👇"),
        gr.Radio(["Image", "Video"], value="Image", label="What do you want?")
    ],
    outputs=gr.File(label="Download Output"),
    title="🖼️🎬 AI Image + Video Generator",
    description="Enter a text prompt and generate an Image or Video with AI."
)

demo.launch(share=True)


✅ Running on: cuda


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 2092 has 14.73 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 219.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
# ==============================================
# AI Image + Video Generator (Google Colab Ready)
# ==============================================

!pip install diffusers transformers accelerate xformers gradio imageio[ffmpeg] -q

import torch, gc, gradio as gr, imageio, numpy as np
from diffusers import AutoPipelineForText2Image, DiffusionPipeline

# Select device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Helper: clear VRAM after each job
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

# ---------------- IMAGE GENERATOR ---------------- #
def generate_image(prompt, steps=20, scale=7.5, height=512, width=512):
    pipe = AutoPipelineForText2Image.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float16
    ).to(device)

    image = pipe(prompt, num_inference_steps=steps, guidance_scale=scale,
                 height=height, width=width).images[0]

    path = "generated_image.png"
    image.save(path)

    del pipe
    clear_memory()
    return path

# ---------------- VIDEO GENERATOR ---------------- #
def generate_video(prompt, steps=25, scale=7.5, frames=16, height=320, width=576):
    pipe = DiffusionPipeline.from_pretrained(
        "damo-vilab/text-to-video-ms-1.7b",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to(device)

    video_frames = pipe(prompt, num_inference_steps=steps, guidance_scale=scale,
                        num_frames=frames, height=height, width=width).frames[0]

    path = "generated_video.mp4"
    imageio.mimsave(path, [np.array(f) for f in video_frames], fps=8)

    del pipe
    clear_memory()
    return path

# ---------------- GRADIO APP ---------------- #
with gr.Blocks() as demo:
    gr.Markdown("# 🎨 AI Image & Video Generator")

    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Enter your prompt")
            mode = gr.Radio(["Image", "Video"], value="Image", label="Mode")
            btn = gr.Button("Generate")
        with gr.Column():
            output_img = gr.Image(label="Generated Image")
            output_vid = gr.Video(label="Generated Video")

    def run(prompt, mode):
        if mode == "Image":
            return generate_image(prompt), None
        else:
            return None, generate_video(prompt)

    btn.click(run, inputs=[prompt, mode], outputs=[output_img, output_vid])

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.8 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc20d9219a42f1922f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
!pip install diffusers transformers accelerate xformers gradio imageio[ffmpeg] -q

import torch, gc, gradio as gr, imageio, numpy as np
from diffusers import AutoPipelineForText2Image, DiffusionPipeline

# Select device
device = "cuda" if torch.cuda.is_available() else "cpu"

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# ---------------- IMAGE GENERATOR ---------------- #
def generate_image(prompt, steps=20, scale=7.5, height=512, width=512):
    pipe = AutoPipelineForText2Image.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float16 if device=="cuda" else torch.float32
    ).to(device)

    image = pipe(prompt, num_inference_steps=steps, guidance_scale=scale,
                 height=height, width=width).images[0]

    path = "generated_image.png"
    image.save(path)

    del pipe
    clear_memory()
    return path

# ---------------- VIDEO GENERATOR ---------------- #
def generate_video(prompt, steps=25, scale=7.5, frames=16, height=320, width=576):
    pipe = DiffusionPipeline.from_pretrained(
        "damo-vilab/text-to-video-ms-1.7b",
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
        variant="fp16"
    ).to(device)

    video_frames = pipe(prompt, num_inference_steps=steps, guidance_scale=scale,
                        num_frames=frames, height=height, width=width).frames[0]

    path = "generated_video.mp4"
    imageio.mimsave(path, [np.array(f) for f in video_frames], fps=8)

    del pipe
    clear_memory()
    return path

# ---------------- GRADIO APP ---------------- #
def run(prompt, mode):
    if mode == "Image":
        img = generate_image(prompt)
        return img, None
    else:
        vid = generate_video(prompt)
        return None, vid

with gr.Blocks() as demo:
    gr.Markdown("# 🎨 AI Image & Video Generator")

    prompt = gr.Textbox(label="Enter your prompt")
    mode = gr.Radio(["Image", "Video"], value="Image", label="Mode")
    btn = gr.Button("Generate")

    output_img = gr.Image(label="Generated Image")
    output_vid = gr.Video(label="Generated Video")

    btn.click(run, inputs=[prompt, mode], outputs=[output_img, output_vid])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0945fb801b0370a31e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
